# LightGBM Naive

In [127]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split, KFold, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import sklearn.metrics as metrics
import librosa

import lightgbm as lgb
from sklearn.utils.class_weight import compute_sample_weight
import optuna

In [128]:
def get_labelname(Tr):
    """[summary]
    training下のラベル名を取得
    Returns:
        [type]: [description]
    """    
    current = os.getcwd()
    if Tr == True:
        filepath = current + '/train/'
    elif Tr == False:
        filepath = current + '/test/'
    print(filepath)
    labellist = []
    for dir in os.listdir(filepath):
        if os.path.isdir(os.path.join(filepath, dir))==True:
            labellist.append(dir)
    
    return labellist, filepath

def wav2list(p):
    """[summary]
    Get audio file list to process all at once
    Returns:
        list : list of audio path
    """
    p = Path(p)
    audio_list = list(p.glob('*.wav'))

    if len(audio_list) == 0:
        sys.exit('Not found in {}'.format(p))

    return audio_list

def get_mfcc_librosa(p):
    """[summary]
    librosaライブラリを用いて24次元MFCCを抽出する
    データはtraining以下に置き, 各ラベルごとにフォルダを作ってデータを置いておく
    Args:
        p ([str]): .wavデータが置いてあるディレクトリ名
    Returns:
        [tupple]: (ファイル名, 24次元のMFCC)
    """    
    wavlist = wav2list(p)
    _name = []
    _mfcc = []
    wavlist.sort()

    for wavfile in wavlist:
        y, sr = librosa.core.load(wavfile,sr=44100)
        tmp = librosa.feature.mfcc(y=y, sr=44100, hop_length=10, win_length=100, n_mfcc=24)
        ceps = tmp.mean(axis=1)
        # print(ceps)
        _name.append(wavfile.stem)
        _mfcc.append(ceps)

    return _name, _mfcc

def make_df(Tr):
    """[summary]
    trainディレクトリ下から.wavデータを取ってきて抽出したMFCCとラベルから構成されるデータフレームを作成
    Returns:
        df_new[dataframe]: 学習データのデータフレーム
    """    
    labellist, filepath = get_labelname(Tr)
    print(labellist)
    cols = [x for x in range(24)]
    print(cols)
    cols.append('label')
    print(cols)
    df_new = pd.DataFrame(index = [], columns=cols)
    # print(df_new)
    for label in labellist:
        # print(filepath, label)
        labelpath = filepath + label
        filename, tmp = get_mfcc_librosa(labelpath)
        df = pd.DataFrame(tmp, index=filename)
        df = df.assign(label=label)
        df_new = pd.concat([df_new, df], axis=0)
    
    return df_new

## 学習データのデータフレーム作成

In [129]:
# Training dataのデータフレーム作成 
df = make_df(Tr=True)
# x : 24次元のMFCC特徴量, y : データのラベル
x = df.iloc[:, 0:24]
y = df.iloc[:, 24]

In [130]:
label = set(y)
label_list = list(label)

for i in range(len(label_list)):
    y[y == label_list[i]] =i

y = np.array(y, dtype = "int")
print(len(y))
y

## テストデータのデータフレーム作成

In [131]:
df_test = make_df(Tr=False)
# x : 24次元のMFCC特徴量, y : データのラベル
x_t = df_test.iloc[:, 0:24]
y_t = df_test.iloc[:, 24]

In [132]:
label_test = set(y_t)
label_list_test = list(label_test)

for i in range(len(label_list_test)):
    y_t[y_t == label_list_test[i]] =i

y_t = np.array(y_t, dtype = "int")
print(len(y_t))
y_t

## LightGBMが扱えるデータにする(Optunaによるパラメータ最適化を施す)

In [133]:
def opt(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 100),
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(x, y, sample_weight=compute_sample_weight("balanced", y))
 
    preds = model.predict(x_t)
    pred_labels = np.rint(preds)
    f1 = f1_score(y_t, pred_labels, average=None)
    return (1.0 - f1.mean())

In [134]:
study = optuna.create_study()
study.optimize(opt, n_trials=100)

In [135]:
study.best_params

In [136]:
fin_lgb = lgb.LGBMClassifier(**study.best_params)
fin_lgb.fit(x, y, sample_weight=compute_sample_weight("balanced", y))

In [146]:
%%timeit
y_pred = fin_lgb.predict(x_t)

## 学習ecopa, テストarena(混合行列はvoices, buzzer, whistle, base)

In [138]:
acc = accuracy_score(y_t, y_pred)
f1_test = f1_score(y_t, y_pred, average=None)
precision_rbf_test = precision_score(y_t, y_pred, average=None)
recall_rbf_test = recall_score(y_t, y_pred, average=None)

print('accuracy : {}'.format(acc))
print("Precision : "+ str(precision_rbf_test))
print("Recall : "+ str(recall_rbf_test))
print('f1: {}'.format(f1_test))
print('f1_mean: {}'.format(f1_test.mean()))

In [139]:
confusion = confusion_matrix(y_t, y_pred)
confusion

## 学習arena, テストecopa(混合行列はvoices, buzzer, whistle, base)

In [125]:
acc = accuracy_score(y_t, y_pred)
f1_test = f1_score(y_t, y_pred, average=None)
precision_rbf_test = precision_score(y_t, y_pred, average=None)
recall_rbf_test = recall_score(y_t, y_pred, average=None)

print('accuracy : {}'.format(acc))
print("Precision : "+ str(precision_rbf_test))
print("Recall : "+ str(recall_rbf_test))
print('f1: {}'.format(f1_test))
print('f1_mean: {}'.format(f1_test.mean()))

In [126]:
confusion = confusion_matrix(y_t, y_pred)
confusion